In [1]:
import pandas as pd


In [5]:
def read_data(file_name):
    data = pd.read_csv(file_name , header=0,
                      delimiter = '\t' , quoting = 3)
    
    return data

In [8]:
train = read_data('labeledTrainData.csv')
test = read_data('testData.csv')
unlabeled_train = read_data('unlabeledTrainData.csv')

In [13]:
train['review'].size + test['review'].size + unlabeled_train['review'].size

100000

In [14]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [15]:
#每一个
def review_to_wordlist(review , remove_stopwords = False):
    #不删除stopwords 因为在强大的word2vec有利用价值
    review_text = BeautifulSoup(review).get_text()
    
    review_text = re.sub("[^a-zA-Z]" , ' ' , review_text)
    
    words = review_text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
    
    return words

In [16]:
import nltk.data

In [21]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [20]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [23]:
def review_to_sentences(review , tokenizer , remove_stopwords = False):
    raw_sentences = tokenizer.tokenize(review.strip())
    #上面的函数 将段落分为句子
    #strip默认将字符串最前面和最后面的空格去掉
    
    #循环处理每一个句子
    sentences = []
    
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence , remove_stopwords))
    
    return sentences

#一个评论是一段
#将一段评论分为一些句子
#每个句子还要再分为一些词

In [24]:
sentences = []

print('training set')
for review in train['review']:
    sentences += review_to_sentences(review , tokenizer)

training set


I:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file I:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
I:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
I:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup

In [26]:
print('unlabel set')
for review in unlabeled_train['review']:
    sentences += review_to_sentences(review , tokenizer)

unlabel set


I:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file I:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
I:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
I:\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should pro

In [30]:
len(sentences)
#看第一条评论包含什么哪些单词

795538

In [31]:
import logging

logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s',
                   level = logging.INFO)


In [32]:
num_features = 300
min_word_count = 400
num_workers = 4
context = 10
downsampling = 1e-3



In [33]:
from gensim.models import word2vec

I:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-03-08 19:14:34,555 : INFO : 'pattern' package not found; tag filters are not available for English


In [34]:
model = word2vec.Word2Vec(sentences , workers=num_workers,
                         size = num_features , min_count = min_word_count,
                         window = context , sample = downsampling)

2018-03-08 19:15:40,603 : INFO : collecting all words and their counts
2018-03-08 19:15:40,607 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-08 19:15:40,715 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-03-08 19:15:40,814 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-03-08 19:15:40,895 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-03-08 19:15:40,980 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-03-08 19:15:41,058 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-03-08 19:15:41,144 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-03-08 19:15:41,223 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-03-08 19:15:41,305 : INFO : PROGRESS: 

2018-03-08 19:15:46,957 : INFO : PROGRESS: at sentence #720000, processed 16105489 words, keeping 118221 word types
2018-03-08 19:15:47,043 : INFO : PROGRESS: at sentence #730000, processed 16331870 words, keeping 118954 word types
2018-03-08 19:15:47,123 : INFO : PROGRESS: at sentence #740000, processed 16552903 words, keeping 119668 word types
2018-03-08 19:15:47,201 : INFO : PROGRESS: at sentence #750000, processed 16771230 words, keeping 120295 word types
2018-03-08 19:15:47,284 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2018-03-08 19:15:47,437 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-03-08 19:15:47,614 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-03-08 19:15:47,734 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-03-08 19:15:47,785 : INFO : collected 123504 word types from a corp

2018-03-08 19:16:42,613 : INFO : EPOCH 2 - PROGRESS: at 93.40% examples, 360298 words/s, in_qsize 7, out_qsize 0
2018-03-08 19:16:43,631 : INFO : EPOCH 2 - PROGRESS: at 95.86% examples, 356787 words/s, in_qsize 7, out_qsize 0
2018-03-08 19:16:44,643 : INFO : EPOCH 2 - PROGRESS: at 99.27% examples, 357439 words/s, in_qsize 8, out_qsize 0
2018-03-08 19:16:44,761 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-03-08 19:16:44,818 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-03-08 19:16:44,823 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-03-08 19:16:44,841 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-03-08 19:16:44,849 : INFO : EPOCH - 2 : training on 17798082 raw words (10938093 effective words) took 30.6s, 357618 effective words/s
2018-03-08 19:16:45,930 : INFO : EPOCH 3 - PROGRESS: at 3.25% examples, 344480 words/s, in_qsize 7, out_qsize 0
2018-03-08 19:16:46,931 : INFO : EPOCH 3 - PR

2018-03-08 19:17:46,556 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-03-08 19:17:46,569 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-03-08 19:17:46,595 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-03-08 19:17:46,604 : INFO : EPOCH - 4 : training on 17798082 raw words (10937294 effective words) took 30.9s, 354001 effective words/s
2018-03-08 19:17:47,687 : INFO : EPOCH 5 - PROGRESS: at 3.59% examples, 385218 words/s, in_qsize 8, out_qsize 0
2018-03-08 19:17:48,695 : INFO : EPOCH 5 - PROGRESS: at 7.60% examples, 407800 words/s, in_qsize 8, out_qsize 0
2018-03-08 19:17:49,712 : INFO : EPOCH 5 - PROGRESS: at 10.59% examples, 378583 words/s, in_qsize 8, out_qsize 0
2018-03-08 19:17:50,724 : INFO : EPOCH 5 - PROGRESS: at 14.62% examples, 391732 words/s, in_qsize 8, out_qsize 1
2018-03-08 19:17:51,745 : INFO : EPOCH 5 - PROGRESS: at 18.98% examples, 405908 words/s, in_qsize 7, out_qsize 0
2018-03-08 19:17:52,757 

In [ ]:
model.init_sims(replace = True)

In [35]:
model_name = '300f_40minword_10window'
model.save(model_name)

2018-03-08 19:19:23,196 : INFO : saving Word2Vec object under 300f_40minword_10window, separately None
2018-03-08 19:19:23,200 : INFO : not storing attribute vectors_norm
2018-03-08 19:19:23,204 : INFO : not storing attribute cum_table
2018-03-08 19:19:23,347 : INFO : saved 300f_40minword_10window


In [47]:
model.doesnt_match('smile cry eat boy'.split())

I:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'boy'

In [53]:
model.most_similar('red')

I:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('blue', 0.6991257667541504),
 ('green', 0.6184930801391602),
 ('snow', 0.5934262275695801),
 ('heat', 0.5785943269729614),
 ('sky', 0.5500898957252502),
 ('wearing', 0.5422793626785278),
 ('pink', 0.5409970283508301),
 ('water', 0.5348068475723267),
 ('tiger', 0.5320825576782227),
 ('eyed', 0.5171416997909546)]

In [55]:
model.most_similar('afraid')

I:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('willing', 0.6274535059928894),
 ('sure', 0.6025159358978271),
 ('convinced', 0.5942209959030151),
 ('aware', 0.5764584541320801),
 ('proud', 0.5758904218673706),
 ('gonna', 0.5232254266738892),
 ('glad', 0.5095299482345581),
 ('ashamed', 0.5074025392532349),
 ('guessing', 0.5048553347587585),
 ('prepared', 0.49442702531814575)]